In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scenedataset.scenedataset import SceneDataset
from torchvision import transforms
from torchvision.transforms import _transforms_video as video_transforms

/root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [36]:
dataset = SceneDataset(paths = ["/SceneDataset/data/02.mkv"], show_progress=True, transform=transforms.Compose([video_transforms.ToTensorVideo(), transforms.Resize((128, 256))]))

2023-02-08 13:51:00.417 | INFO     | scenedataset.scenedataset:_get_video_paths:448 - Found 1 videos


In [42]:
!pip install pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 kB 10.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [4]:
from torchmetrics import (
    MeanAbsoluteError,
    MeanSquaredError,
    StructuralSimilarityIndexMeasure,
)
mae = MeanAbsoluteError()
mse = MeanSquaredError()
ssim = StructuralSimilarityIndexMeasure()

In [5]:
from ignite.metrics import SSIM, MeanSquaredError, FID

In [9]:
# ssim = SSIM(data_range=1.0)
# ssim = MeanSquaredError()
fid = FID(64)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Loading model from: /root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth


In [37]:
from torchvision.utils import save_image
from ignite.engine import *
import matplotlib.pyplot as plt
import lpips


# def eval_step(engine, batch):
#     return batch

# default_evaluator = Engine(eval_step)

# fid.attach(default_evaluator, 'fid')

import torch
import numpy as np
from torchmetrics.image.fid import FrechetInceptionDistance


loss_fn_vgg = lpips.LPIPS(net='vgg')


for i in range(0, len(dataset)):
    frames = dataset[i]    
    frames = frames.permute(1, 0, 2, 3)

    for j in range(0, len(frames) - 1):
        current_frame = frames[j : j + 1]
        next_frame = frames[j + 1 : j + 2]
        
        # frames is between 0 and 1, scale between -1 and 1
        difference = loss_fn_vgg(current_frame * 2 - 1, next_frame * 2 -1).item()

        # save_image(current_frame, f"./images/current_frame_{i}_{j}.png")
        # save_image(next_frame, f"./images/next_frame_{i}_{j}.png")
        # plt.imsave(f"./images/current_frame_{i}_{j}.png", current_frame[0].permute((1, 2, 0)).numpy())
        # plt.imsave(f"./images/next_frame_{i}_{j}.png", next_frame[0].permute((1, 2, 0)).numpy())
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_axes([0,0,1,1])
        ax.imshow(current_frame[0].permute((1, 2, 0)).numpy())
        ax.set_title(f"fid: {difference}")
        plt.savefig(f"./images/{difference:.5f}_current_frame_{i}_{j}.png")
        plt.close()

        
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_axes([0,0,1,1])
        ax.imshow(next_frame[0].permute((1, 2, 0)).numpy())
        ax.set_title(f"fid: {difference}")
        plt.savefig(f"./images/{difference:.5f}_next_frame_{i}_{j}.png")
        plt.close()

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /root/.cache/pypoetry/virtualenvs/scenedataset-mZH_Fr2s-py3.8/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth


KeyboardInterrupt: 

In [38]:
import imageio
from glob import glob
import re

filenames = glob("./images/*_current_frame_*.png")

for filename in filenames:
    
    try:
        difference, i, j = re.findall(r"(\d+.\d+)_current_frame_(\d+)_(\d+).png", filename)[0]
    except:
        print(filename)
    images = []
    images.append(imageio.imread(f"./images/{difference}_current_frame_{i}_{j}.png"))
    images.append(imageio.imread(f"./images/{difference}_next_frame_{i}_{j}.png"))
    imageio.mimsave(f"./video/{difference}_animation_{i}_{j}.gif", images)



/tmp/ipykernel_1591198/2186432617.py:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"./images/{difference}_current_frame_{i}_{j}.png"))
/tmp/ipykernel_1591198/2186432617.py:15: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"./images/{difference}_next_frame_{i}_{j}.png"))


In [7]:
broken_dataset = SceneDataset(paths = ["/SceneDataset/data/02.mkv"], show_progress=True, min_max_len=(15, 25), duplicate_threshold=0.1, duplicate_method="ssim")

2023-02-07 07:34:27.185 | INFO     | scenedataset.scenedataset:_get_video_paths:445 - Found 1 videos
2023-02-07 07:34:27.193 | INFO     | scenedataset.scenedataset:retrieve_scenes_from_video:235 - Removing duplicate scenes...


Loading scenes to find duplicates...:   0%|          | 0/25 [00:00<?, ?it/s]

[25]


Loading scenes to find duplicates...:   0%|          | 0/54 [00:00<?, ?it/s]

[54]


Loading scenes to find duplicates...:   0%|          | 0/20 [00:00<?, ?it/s]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
